In [9]:
from drn_interactions.config import Config, ExperimentInfo
from drn_interactions.io import load_derived_generic
from drn_interactions.stats import se_mean


In [2]:
sessions = ExperimentInfo.foot_shock_sessions_10min

In [3]:
len(sessions)

8

In [4]:
neurons = load_derived_generic("neuron_types.csv").query('session_name in @sessions').dropna()

In [5]:
len(neurons)

259

In [16]:

by_session = neurons.groupby("session_name").neuron_id.count()
by_session.mean().round(2), se_mean(by_session).round(2)

(32.38, 4.67)

In [10]:
neurons["neuron_type"].value_counts()

SIR    118
SR     111
FF      30
Name: neuron_type, dtype: int64

In [22]:
from drn_interactions.stats import se_mean

neurons.groupby('session_name')["neuron_type"].value_counts().unstack().apply(lambda x: f"{x.mean():.2f} + {se_mean(x):.2f}")

neuron_type
FF      3.75 + 1.06
SIR    14.75 + 4.01
SR     13.88 + 2.47
dtype: object

## Fast FS

In [12]:
from drn_interactions.io import load_derived_generic
from drn_interactions.stats import se_mean, se_prop
import numpy as np

df = load_derived_generic("fast_fs_foot_shock_unit_responders.csv")

In [19]:
import pandas as pd

def prop_responses(df):
    out = {}
    for response in ["activated", "inhibited", "non-responder"]:
        col = (df["response"] == response)
        out[response] = f"{col.mean() * 100:.2f} +- {se_prop(col) * 100:.2f}"
    return pd.Series(out)

(
    df
    .assign(
        response=lambda x: np.where(
            x["sig"], np.where(x["Diff"] > 0, "activated", "inhibited"), 
            "non-responder"
            )
        )
    .groupby("neuron_type")
    .apply(prop_responses)
    .loc[["SR", "SIR", "FF"]]
)



,activated,inhibited,non-responder
neuron_type,,,
SR,25.23 +- 4.12,39.64 +- 4.64,35.14 +- 4.53
SIR,49.15 +- 4.60,16.95 +- 3.45,33.90 +- 4.36
FF,33.33 +- 8.61,33.33 +- 8.61,33.33 +- 8.61


## Slow TS

In [21]:
df = load_derived_generic("slow_ts_foot_shock_unit_responders_pre_to_shock.csv")

In [23]:
(
    df
    .assign(
        response=lambda x: np.where(
            x["sig"], np.where(x["diff_inv"] > 0, "activated", "inhibited"), 
            "non-responder"
            )
        )
    .groupby("neuron_type")
    .apply(prop_responses)
    .loc[["SR", "SIR", "FF"]]
)

,activated,inhibited,non-responder
neuron_type,,,
SR,56.76 +- 4.70,35.14 +- 4.53,8.11 +- 2.59
SIR,57.63 +- 4.55,33.05 +- 4.33,9.32 +- 2.68
FF,43.33 +- 9.05,46.67 +- 9.11,10.00 +- 5.48


## Slow Post

In [24]:
df = load_derived_generic("slow_ts_foot_shock_unit_responders_pre_to_post.csv")

In [25]:
(
    df
    .assign(
        response=lambda x: np.where(
            x["sig"], np.where(x["diff_inv"] > 0, "activated", "inhibited"), 
            "non-responder"
            )
        )
    .groupby("neuron_type")
    .apply(prop_responses)
    .loc[["SR", "SIR", "FF"]]
)

,activated,inhibited,non-responder
neuron_type,,,
SR,11.71 +- 3.05,76.58 +- 4.02,11.71 +- 3.05
SIR,16.10 +- 3.38,68.64 +- 4.27,15.25 +- 3.31
FF,23.33 +- 7.72,63.33 +- 8.80,13.33 +- 6.21
